In [2]:
pip install chromedriver-autoinstaller

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [3]:
import chromedriver_autoinstaller
from selenium import webdriver
import time

class Driver:
    @staticmethod
    def driver_init():
        chromedriver_autoinstaller.install()
        options = webdriver.ChromeOptions()
#         options.add_argument('headless') # jupyter notebook 환경에서 주석처리
        options.add_argument('lang=ko_KR')
        options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36")
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-notifications')
        options.add_argument('--disable-popup-blocking')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--disable-browser-side-navigation')
        options.add_argument('disable-gpu')

        driver = webdriver.Chrome(options=options)
        driver.set_window_size(1600, 1100)

        return driver

In [20]:
driver = Driver.driver_init()

In [25]:
urls = [
        ('https://frizm.co.kr/product/list.html?cate_no=24',
         'top', 17), #아우터
        ('https://frizm.co.kr/product/list.html?cate_no=52',
         'top', 37), #상의
        ('https://frizm.co.kr/product/list.html?cate_no=26',
         'bottom', 14), #하의
         ]

base_dict= {"baseCat":[], "baseName": [], "baseCode": [], "baseUrl": [], "colorName": [],
            "baseGender": [], "price": [], "basePrice": [], "colorFabric": [], "imageUrl": [],
            "size": [], "isAvailable": []
            }

for (url,cat, pages_cnt) in urls:
    for page_num in range(pages_cnt):
        try:
            driver.get(url + "&page=" + str(page_num))
        except:
            continue

        product_paths = []

        products = driver.find_elements('xpath', '//ul[@class="prdList grid4"]/li')
        products_cnt = len(products)

        ## 각 product path를 product_paths 리스트에 저장
        for i in range(products_cnt):
            product = driver.find_elements('xpath', '//ul[@class="prdList grid4"]/li')[i]
            product_path = product.find_element('xpath', './div[@class="thumbnail"]/a').get_attribute('href')
            product_paths.append(product_path)



        for i in range(products_cnt):
            product_path = product_paths[i]

            driver.get(product_path)
            time.sleep(2)

            baseURL = product_path
            base_gender = 0
            text_info = driver.find_element('xpath', '//div[@class="infoArea"]')
            imageURL = driver.find_element('xpath','//img[@class="BigImage "]').get_attribute('src')

            base_name = text_info.find_element('xpath', './div[@class="headingArea"]/h2').text
            base_code = base_name.split("_")[0]
            color_name = base_name.split("_")[1]
            base_price = text_info.find_element("xpath",'.//strong[@id="span_product_price_text"]').text
            
            try:
                price = text_info.find_element("xpath",'.//span[@id="span_product_price_sale"]').text
                color_fabric = text_info.find_element("xpath",'./div[2]/table/tbody/tr[5]/td/span').text
            except:
                price = base_price
                color_fabric = text_info.find_element("xpath",'./div[2]/table/tbody/tr[4]/td/span').text

            size_elements = text_info.find_elements("xpath",'.//optgroup[@label="사이즈"]/option')
            size = "M"
            is_available = True

            for element in size_elements:
                size = element.text.replace(" [품절]","")
                if "[품절]" in element.text:
                    is_available = False
                else:
                    is_available = True

                base_dict["baseCat"].append(cat)
                base_dict["baseName"].append(base_name)
                base_dict["baseCode"].append(base_code)
                base_dict["baseUrl"].append(baseURL)
                base_dict["colorName"].append(color_name)
                base_dict["baseGender"].append(0)
                base_dict["price"].append(price)
                base_dict["basePrice"].append(base_price)
                base_dict["colorFabric"].append(color_fabric)
                base_dict["imageUrl"].append(imageURL)
                base_dict["size"].append(size)
                base_dict["isAvailable"].append(is_available)
        print(len(base_dict["baseCat"]))


120
240
360
483


KeyboardInterrupt: 